# Contextual bandit

콘텍스트 밴딧에서는 상태(state)개념을 도입한다. 이를 통해 어떤 밴딧을 다루고 있는지 알 수 있다. </br>
다시말해 하나의 밴딧만 다루던 문제에서 여러 밴딧을 다루며 에이전트의 목표는 단지 1개의 밴딧이 아니라 여러 개의 밴딧에 대해 최선의 액션을 학습하는 것으로 확장되었다. </br>
contextual: 맥락[전후 사정]과 관련된

In [11]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [8]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        # 밴딧들의 손잡이 목록을 작성(행: 밴딧, 열: 밴딧의 손잡이(액션))
        # 동일 행에서 각각의 칼럼은 밴딧의 손잡이
        self.bandits = np.array([[0.2, 0, -0.1], 
                                 [0.4, 0.0, -0.2], 
                                 [-1.1, 1.2, 3.0]])
        self.num_bandits = self.bandits.shape[0] # 밴딧의 개수 설정(행)
        self.num_actions = self.bandits.shape[1] # 밴딧의 액션 개수 설정(열)
    
    def getBandit(self):
        # 각각의 에피소드에 대해 랜덤한 상태를 반환
        self.state = np.random.randint(0, self.num_bandits)
        
        return self.state
    
    def pullArm(self, action):
        # 랜덤한 수를 얻는다. 
        bandit = self.bandits[self.state, action]
        result = np.random.randn(1)
        if result > bandit:
            return 1
        else:
            return -1

In [9]:
# 에이전트 클래스
# 정책 경사 방법을 사용해 선택된 액션에 대해 더 큰 보상을 받는 쪽으로 이동하도록 에이전트를 업데이트

class agent():
    def __init__(self, lr, s_size, a_size):
        # 네트워크의 피드포워드 부분, 에이전트는 상태를 받아서 액션을 출력
        self.state_in = tf.placeholder(shape=[1], dtype=tf.uint8)
        state_in_OHE = slim.one_hot_encoding(self.state_in, s_size)
        output = slim.fully_connected(state_in_OHE, a_size, biases_initializer=None, 
                                      activation_fn=tf.nn.sigmoid, weights_initializer=tf.ones_initializer())
        # output 차원을 1차원 벡터로 변환
        self.output = tf.reshape(output, [-1])
        # output에서 가장 큰 값 추출
        self.chosen_action = tf.argmax(self.output, axis=0)
        
        # 학습과정을 구현
        # 비용을 계산하기 위해 봇아과 선택된 액션을 네트워크에 피드하고
        # 네트워크를 업데이트하는 데에 이를 이용한다. 
        self.reward_holder = tf.placeholder(shape=[1], dtype=tf.float32) # 텐서플로우는 float16지원 안함
        self.action_holder = tf.placeholder(shape=[1], dtype=tf.int32) # 여기서는 uint8지원 안함, 아래 tf.slice에서 begin인자가 int32이상을 받음
        self.responsible_weight = tf.slice(input_=self.output, begin=self.action_holder, size=[1]) #self.output을 받아 self.action_holder부터 [1]사이즈만큼 슬라이스
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

In [10]:
# 에이전트 학습
# 에이전트는 환경의 상태를 알아내고, 액션을 취하고, 보상을 받으면서 학습

# 텐서플로우 그래프 리셋
tf.reset_default_graph()
# 밴딧 로드
cBandit = contextual_bandit()
# 에이전트 로드
myAgent = agent(lr=0.001, s_size=cBandit.num_bandits, a_size=cBandit.num_actions)
# 네트워크 내부를 들여다보기 위해 평가할 가중치
weights = tf.trainable_variables()[0]

# 에이전트를 학습시킬 전체 에피소드 수 설정
total_episodes = 10000
# 밴딧에 대한 점수판을 0으로 설정
total_reward = np.zeros([cBandit.num_bandits, cBandit.num_actions])
# 랜덤한 액션을 취할 가능성 설정
e = 0.1

init = tf.global_variables_initializer()

# 텐서플로우 그래프 런칭
with tf.Session() as sess:
    sess.run(init)
    for i in range(total_episodes):
        s = cBandit.getBandit()
        # 네트워크로부터 랜덤한 액션 또는 하나의 액션을 선택한다. 
        if np.random.rand(1) < e: # e로 설정한 아주 작은 수보다 작을 경우 랜덤한 액션 선택
            action = np.random.randint(cBandit.num_actions)
        else: # 아닌 경우 상태를 받아서 액션 선택
            action = sess.run(myAgent.chosen_action, feed_dict={myAgent.state_in:[s]})
            
        # 주어진 밴딧에 대해 액션을 취한 보상을 얻는다.
        reward = cBandit.pullArm(action)
        
        # 네트워크를 업데이트
        feed_dict = {myAgent.reward_holder:[reward], myAgent.action_holder:[action], myAgent.state_in:[s]}
        _, ww = sess.run([myAgent.update, weights], feed_dict=feed_dict)
        
        # 보상의 총계 업데이트
        total_reward[s, action] += reward
        
        if not i % 1000:
            print("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward, axis=1)))
            print("The best actions for each bandit are " + str(np.argmax(ww[0])+1), str(np.argmax(ww[1])+1), str(np.argmax(ww[2])+1))
    
# 결과값 출력
for bandit in range(cBandit.num_bandits):
    print(str(bandit) + " bandit's best action is " + str(np.argmax(ww[bandit])+1))
    if np.argmax(ww[bandit]) == np.argmin(cBandit.bandits[bandit]):
        print("It is correct")
    else:
        print("It is wrong!...")

Mean reward for each of the 3 bandits: [ 0.          0.         -0.33333333]
The best actions for each bandit are 1 1 1
Mean reward for each of the 3 bandits: [15.66666667 15.33333333 76.66666667]
The best actions for each bandit are 3 3 1
Mean reward for each of the 3 bandits: [ 24.66666667  33.         144.66666667]
The best actions for each bandit are 3 3 1
Mean reward for each of the 3 bandits: [ 32.66666667  43.66666667 224.66666667]
The best actions for each bandit are 3 3 1
Mean reward for each of the 3 bandits: [ 40.33333333  52.33333333 291.66666667]
The best actions for each bandit are 3 3 1
Mean reward for each of the 3 bandits: [ 54.          59.66666667 356.66666667]
The best actions for each bandit are 3 3 1
Mean reward for each of the 3 bandits: [ 59.66666667  73.         425.        ]
The best actions for each bandit are 3 3 1
Mean reward for each of the 3 bandits: [ 63.33333333  89.         492.66666667]
The best actions for each bandit are 3 3 1
Mean reward for each o